In [1]:
%matplotlib tk
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os, glob, cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '../dataset/images-full/'
classes = ["100", "106", "109", "118", "213", "217"]

In [3]:
def load_data_cnn(train_path):
    images = []
    labels = []
    classes = [lists for lists in os.walk(train_path)][0][1]    
    for cls in classes:
        file_path = os.path.join(train_path, cls, "*g")
        for file in glob.glob(file_path):
            image = cv2.imread(file)
            image = image[288-250:288-35, 55:390]
            image = cv2.resize(image, (int(215/5), int(215/5)), cv2.INTER_LINEAR)
            label = np.zeros(len(classes))
            label[classes.index(cls)] = 1.0
            images.append(image)
            labels.append(label) 
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

dataset = load_data_cnn(train_path)

0 - normal <br>
1 - pvc <br>
2 - lbbb <br>
3 - rbbb <br>
4 - fusion <br>
5 - paced

In [4]:
X_train = []
X_train.append(dataset[0][:80])
X_train.append(dataset[0][100:180])
X_train.append(dataset[0][200:280])
X_train.append(dataset[0][300:380])
X_train.append(dataset[0][400:480])
X_train.append(dataset[0][500:580])
X_train = [j for i in X_train for j in i]
X_train = np.array(X_train)

y_train = []
y_train.append(dataset[1][:80])
y_train.append(dataset[1][100:180])
y_train.append(dataset[1][200:280])
y_train.append(dataset[1][300:380])
y_train.append(dataset[1][400:480])
y_train.append(dataset[1][500:580])
y_train = [j for i in y_train for j in i]
y_train = np.array(y_train)

X_test = []
X_test.append(dataset[0][80:100])
X_test.append(dataset[0][180:200])
X_test.append(dataset[0][280:300])
X_test.append(dataset[0][380:400])
X_test.append(dataset[0][480:500])
X_test.append(dataset[0][580:600])
X_test = [j for i in X_test for j in i]
X_test = np.array(X_test)

y_test = []
y_test.append(dataset[1][80:100])
y_test.append(dataset[1][180:200])
y_test.append(dataset[1][280:300])
y_test.append(dataset[1][380:400])
y_test.append(dataset[1][480:500])
y_test.append(dataset[1][580:600])
y_test = [j for i in y_test for j in i]
y_test = np.array(y_test)

y_test_label = []
for i in range(120):
    if i >= 100:
        y_test_label.append(5)
    elif i >= 80:
        y_test_label.append(4)
    elif i >= 60:
        y_test_label.append(3)
    elif i >= 40:
        y_test_label.append(2)
    elif i >= 20:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
        

In [5]:
input_shape = (int(215/5), int(215/5), 3)

In [6]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3, 3), input_shape=input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3)))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3)))
cnn_model.add(BatchNormalization())
cnn_model.add(Activation('relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(6, activation='softmax'))

In [7]:
cnn_model.compile(optimizer=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
classifier = cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=50, batch_size=18, shuffle=True)

Train on 432 samples, validate on 48 samples
Epoch 1/50
432/432 [==============================] - 2s 4ms/step - loss: 1.8225 - acc: 0.3796 - val_loss: 2.0538 - val_acc: 0.0000e+00
Epoch 2/50
432/432 [==============================] - 0s 605us/step - loss: 1.0172 - acc: 0.6435 - val_loss: 1.4372 - val_acc: 0.5000
Epoch 3/50
432/432 [==============================] - 0s 581us/step - loss: 0.7338 - acc: 0.7292 - val_loss: 1.2305 - val_acc: 0.6667
Epoch 4/50
432/432 [==============================] - 0s 675us/step - loss: 0.5422 - acc: 0.8148 - val_loss: 0.8282 - val_acc: 0.8542
Epoch 5/50
432/432 [==============================] - 0s 592us/step - loss: 0.4457 - acc: 0.8611 - val_loss: 0.6017 - val_acc: 0.8958
Epoch 6/50
432/432 [==============================] - 0s 580us/step - loss: 0.3597 - acc: 0.8796 - val_loss: 0.5816 - val_acc: 0.8750
Epoch 7/50
432/432 [==============================] - 0s 582us/step - loss: 0.2705 - acc: 0.9259 - val_loss: 0.7642 - val_acc: 0.7708
Epoch 8/50
432/

In [9]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print("Loss = "+str(loss))
print("Accuracy = "+str(accuracy))

120/120 [==============================] - 0s 307us/step
Loss = 0.014960017551978429
Accuracy = 0.9916666666666667


In [16]:
acc_values = cnn_model.history.history['acc']
val_acc_values = cnn_model.history.history['val_acc']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'r', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [11]:
predictions = cnn_model.predict_classes(X_test, batch_size=5)

In [12]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 1, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [13]:
cn = confusion_matrix(y_test_label, predictions)

In [14]:
print ("Confusion Matrix")
cn

Confusion Matrix


array([[20,  0,  0,  0,  0,  0],
       [ 0, 20,  0,  0,  0,  0],
       [ 0,  0, 20,  0,  0,  0],
       [ 0,  1,  0, 19,  0,  0],
       [ 0,  0,  0,  0, 20,  0],
       [ 0,  0,  0,  0,  0, 20]])

In [15]:
cnn_model.save("../models/cnn_6labels_1.h5")